#### HMA1 Class 
This is a demo notebook to discover functionalities of the HMA1 class for relational data modelling.

#### What is HMA1?
The bulian.relational.HMA1 class implements a Hierarchical Modeling Algorithm which is an algorithm that allows to recursively walk through a relational dataset and applies tabular models across all the tables in a way that lets the models learn how all the fields from all the tables are related.

#### Import all the relevant packages

In [15]:
import os,sys
import pandas as pd
from bulian.metrics.demo import sample_relational_demo
from bulian.metrics.reports import get_multi_table_report
from bulian.utils import display_tables
from bulian.relational import HMA1

#### Start by Loading Bulian Demo relational dataset

In [16]:
metadata, tables = sample_relational_demo()

In [17]:
metadata

Metadata
  root_path: .
  tables: ['users', 'sessions', 'transactions']
  relationships:
    sessions.user_id -> users.user_id
    transactions.session_id -> sessions.session_id

##### This returns two objects, the relational data tables and the metadata associated with each table 

In [18]:
print(metadata)

Metadata
  root_path: .
  tables: ['users', 'sessions', 'transactions']
  relationships:
    sessions.user_id -> users.user_id
    transactions.session_id -> sessions.session_id


In [35]:
for name, table in tables.items():
    print(name, table.shape)

users (30, 4)
sessions (43, 5)
transactions (226, 5)


In [36]:
display_tables(tables)

#### Deep dive into metadata class

- Get Tables to view associated table names
- Get parents(table) to view child-parent releationship for a particular table
 

In [37]:
metadata.get_tables()

['users', 'sessions', 'transactions']

In [38]:
metadata.get_parents('sessions')

{'users'}

#### Next, lets fit a HMA1 model to learn this data to eventually sample synthetic data rows

- Import bulian.relational.HMA1 and create an instance of it passing relevant metadata
- Fit the instance to the tables dict

In [19]:
model = HMA1(metadata)
model.fit(tables)

*The fit method basically iterates through all the datasets in the tables dict and models each dataset using Gaussian Copula models*  

#### Next, sample synthetic data rows from the fitted HMA1 model 

In [20]:
new_data = model.sample(num_rows=100,)

Sampling rows: 100%|██████████| 5/5 [00:00<00:00, 545.38it/s]


In [41]:
display_tables(new_data)

In [42]:
for name, table in new_data.items():
    print(name, table.shape)

users (100, 4)
sessions (164, 5)
transactions (771, 5)


#### Generate Report on Real and Synthetic Data

After new data has been generated you can create a quality report on the same. The function takes in the real data, synthetic data and metadata as required input parameters. You can also specific the numeric and discrete columns explicitly or let the function determine it accordingly. 

The inputs for numeric and discrete columns is dictionary with the key name being the table name and the values is a list of valid column names present in the tables.

##### Generate Report without specifying columns

In [9]:
get_multi_table_report(tables, new_data, metadata)

/home/anurag/Tabular_Synthesizers/bulian/metrics/utils.py:82: UserWarning: Unexpected value nan in synthetic data.
  warnings.warn(f'Unexpected value {value} in synthetic data.')


##### Generate Report with defined columns

As mentioned above you can also specify the numeric and discrete columns and pass them as parameters while generating the report.

In [1]:
discrete_columns = {
    'users': ['country', 'gender'],
    'sessions': ['device', 'os'],
    'transactions': ['timestamp', 'cancelled']
}

numeric_columns = {
    'users': ['age'],
    'sessions': ['minutes'],
    'transactions': ['amount']
}

In [22]:
get_multi_table_report(tables, new_data, metadata, discrete_features=discrete_columns, numeric_features=numeric_columns)

/home/anurag/Tabular_Synthesizers/bulian/metrics/utils.py:82: UserWarning:

Unexpected value nan in synthetic data.



#### Save and share the model:  
Once you have fitted the model, all you need to do is call its save method passing the name of the file in which you want to save the model. Note that the extension of the filename is not relevant, but we will be using the .pkl extension to highlight that the serialization protocol used is pickle.

In [43]:
model.save('relational_model.pkl')

**Important** If you inspect the generated file you will notice that its size is much smaller than the size of the data that you used to generate it. This is because the serialized model contains **no information about the original data**, other than the parameters it needs to generate synthetic versions of it. This means that you can safely share this `relational_model.pkl` file without the risk of disclosing any of your real data!

#### Sample only a subset of tables? 

In some occasions you will not be interested in generating rows for the entire dataset and would rather generate data for only one table and its children.
To do this you can simply pass the name of the table that you want to sample.
For example, pass the name sessions to the sample method, the model will only generate data for the sessions table and its child table, transactions.

In [44]:
model.sample('sessions', num_rows=25)

Sampling rows: 100%|██████████| 17/17 [00:00<00:00, 81.73it/s]


{'sessions':     session_id  user_id  device       os  minutes
 0          164      105  tablet  windows       51
 1          165      101      pc    linux       44
 2          166      112  tablet  android       26
 3          167      103      pc  android       43
 4          168      107      pc  android       56
 5          169      112  tablet    linux       46
 6          170      107      pc  android       46
 7          171      100      pc    linux       60
 8          172      107  mobile  android       38
 9          173      110      pc  android       48
 10         174      107  tablet    linux       21
 11         175      103  tablet  android       29
 12         176      105      pc    linux       47
 13         177      101      pc    linux       57
 14         178      107      pc  android       46
 15         179      105      pc    linux       45
 16         180      100  mobile  android       29
 17         181      113      pc  android       62
 18         182    

If you want to further restrict the sampling process to only one table and also skip its child tables, you can add the argument sample_children=False. For example, you can sample data from the table users only without producing any rows for the tables sessions and transactions.

In [45]:
model.sample('users', num_rows=5, sample_children=False)

Sampling rows: 100%|██████████| 5/5 [00:00<00:00, 23.92it/s]


,user_id,country,gender,age
0,117,MM,F,22
1,118,KN,F,32
2,119,PK,F,30
3,120,MV,F,26
4,121,MV,F,32


**Note** In this case, since we are only producing a single table, the output is given directly as a `pandas.DataFrame` instead of a dictionary.

### Fin ###